In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tifffile
from skimage import filters
from scipy.ndimage import label, binary_closing

from pathlib import Path
from PIL import Image
import imageio
import cv2

from patchnmf.data_io import *
from patchnmf.analyse.videography_compute import *
from patchnmf.analyse.videography_plot import *

In [ ]:
def mkdir(path):
    if not os.path.exists(path): os.makedirs(path)
    return path

In [ ]:
data_path = rf'C:\Users\zaggila\Documents\pixelNMF\data' 
# sessions = sorted([f for f in os.listdir(data_path) if f.endswith('_cell_control')])
# print(f'All sessions: {sessions}')

ds = 'sz89_2024-06-06_a_cell_control'

subject_path = os.path.join(data_path, ds) # choose one specifc ds 
movie_path = os.path.join(subject_path, 'cam_crop.tif')

In [ ]:
# load uncompressed tiff

tiff = io.imread(movie_path, plugin='tifffile') #pil loads on snapshot 

# making sure smallest value of tiff is zero - just a linear transform, shouldn't affect NMF ? 
tiff -= np.min(tiff)
print(f'Shape of video: {tiff.shape}')

In [ ]:
# LossLess compression (output tiff or avi)

# Normalize if needed
tiff = tiff - np.min(tiff)
tiff = (tiff / tiff.max() * 255).astype(np.uint8)

# Convert grayscale to BGR if needed
if len(tiff.shape) == 3:  # (frames, height, width)
    tiff = np.stack([tiff] * 3, axis=-1)

# Set up AVI writer
height, width = tiff.shape[1:3] 
out = cv2.VideoWriter(
    'compressed_beh_movie.avi',
    cv2.VideoWriter_fourcc(*'FFV1'),  # Lossless codec (e.g., FFV1, MJPG, or XVID for near-lossless)
    15,  # FPS
    (width, height)
)

# Write frames
for frame in tiff:
    out.write(frame)
out.release()
print("Saved to output_video.avi") # save compressed avi to load in GUI 